<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# <font color="#76b900">**1:** 大语言模型入门</font>

**欢迎学习本课程！** 这是本课程的第一个 notebook，旨在带您熟悉载入 LLM 的工作流程，了解我们所面对的问题、资源和目标！

#### 学习目标：

* 回顾有关深度学习的一些基本假设，了解它们如何扩展到语言建模。
* 将您的首个 LLM 加载到环境中，探究其架构，并观察它是如何执行的！



---




## 1.1. 回顾深度学习

在深度学习的整个学习过程中，您可能已经在分类和回归任务上优化过了几种模型。您可能已经经历了以下几个阶段：

* 刚开始时，您会用**线性回归**和**逻辑回归**来为输入和输出之间的简单线性关系进行建模和解释。
* 当这依然不足以解决问题时，您开始逐层**堆叠线性层**，并添加**非线性激活函数**，以增强模型的预测能力。
* 当遇到难以解决的高维度数据时，您开始使用像**卷积这类稀疏连接技术**来更好的控制推理。
* 当您意识到没有足够的数据来为每个特定任务训练模型时，您学会了使用**预训练组件（如 VGG-16/ResNet）**，它们已经在巨大的训练数据集上进行过训练，具备了您所需要的基本能力。

> <div><img src="imgs/machine-learning-process.jpg" width="800"/></div>
>
> **来源: [High-Performance Data Science with RAPIDS | NVIDIA](https://www.nvidia.com/en-us/deep-learning-ai/software/rapids/)**

如果您已经完成了所有这些课程，那么恭喜！您已经具备了进阶的条件，是时候迈向更深更广的领域，例如语言建模了！

与视觉类似，如果不认真对待，语言将是一个极其复杂的高维度领域。回想一下，一张常见的 200x200 彩色图像包含 $200\times 200\times 3 = 120,000$ 个特征！再想像一下，一句话里会包括多少种词的组合？**只能说很多！** 幸运的是，我们有大量创造性的技术使这个问题变得可以处理，大型预训练模型的生态系统就有能实现它的各种工具！

**这就是本课程要讲的：如何解决语言建模，有哪些可用的工具，以及会面临些什么问题！**



---




## 1.2. 获取我们的第一个 LLM

本课程不会带您从 0 开始搭建模型，而是向您介绍可以使用的最佳工具，并在必要的时候带您深入了解它们是如何工作的。开启我们语言建模之旅的最佳工具是： **HuggingFace &#x1F917;!**

[**HuggingFace**](https://huggingface.co/) 是一个开源社区，提供简单的方法用于访问、上传和测试及部署大型深度学习模型。它支持多种任务和模态，但在本课程中，我们将重点关注大语言模型 (**LLMs**).

在搜索 [HuggingFace 模型目录](https://huggingface.co/models?sort=downloads&search=bert)时，您很快就会发现 [`bert-base-uncased`](https://huggingface.co/bert-base-uncased) 模型。看看它的介绍卡片，您会看到几个有趣的东西：

1. 加载模型会用到 [`transformers`](https://github.com/huggingface/transformers) 包。这是 HuggingFace 用来支持平台上大多数语言模型代码的包。它的名称，`transformers` 指的是诸多此类模型背后的主要架构，我们将在下一个 notebook 中详细介绍此结构。可以先熟悉一下 `transformers` ，之后我们就会用到它，您可以随时搜索并深入了解它的源代码！
2. 这个卡片还写了一个可以用于填充掩码（稍后会讨论）的默认版本，可以通过它的 [Pipelines](%5Bhttps://huggingface.co/docs/transformers/main_classes/pipelines%5D) 来实现。这里的 **pipeline** 指的是从人可读懂的输入到人可读懂的输出的端到端的流程。这会让使用模型变得非常简单，甚至会让你忘记在背后的某个地方正进行着输入/输出张量的可微分过程。

作为一个有代表性的例子，我们先来把刚刚讨论过的 [`bert-base-uncased`](https://huggingface.co/bert-base-uncased) 模型拉下来试试！

In [ ]:
from transformers import pipeline

## Loading in the pipeline and predict the mask fill (example from model card)
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Hello I'm a [MASK] model.")

看，它起作用了！在背后，有个深度学习模型在某个地方，吞下一堆数字并给出概率来让这一切实现。但有时候很容易忘记这一点，尤其是当您用的模型正在像人一样回复文字时，您可能会开始怀疑它是否正连着加利福尼亚州某个数据中心的人类大脑。本课程的目的，正是**探究背后究竟发生了什么，并知道如何用它来搭建好用的系统。**



---




## 1.3. 剖析 Pipeline

只像我们刚刚那样，看到 pipeline 拿到字符串之后再给出字典，对我们的理解没什么帮助，所以现在我们来看看 pipeline 里面到底做了什么。我们可以从接口的抽象稍微向下走一层，观察 pipeline 内部的结构：

In [ ]:
from transformers import AutoTokenizer, BertTokenizer, BertModel, FillMaskPipeline, AutoModelForMaskedLM, BertForMaskedLM, BertForPreTraining

from transformers import AutoTokenizer, AutoModel        ## General-purpose fully-automatic
from transformers import AutoModelForMaskedLM            ## Default import for FillMaskPipeline
from transformers import BertTokenizer, BertForMaskedLM  ## Realized components after automatic resolution

class MyMlmPipeline(FillMaskPipeline):
    def __init__(self):
        ## The fully-automatic version
        super().__init__(
            tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased'),
            model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
        )

    def __call__(self, string, verbose=False):
        ## Verbose argument just there for our convenience
        input_tensors = self.preprocess(string)
        if verbose: print('\npreprocess outputs:\n', input_tensors, '\n')
        output_tensors = self.forward(input_tensors)
        if verbose: print('forward outputs:\n', output_tensors, '\n')
        output = self.postprocess(output_tensors)
        return output

    # def preprocess(self, string):
    #     string = [string] if isinstance(string, str) else string
    #     inputs = self.tokenizer(string, return_tensors="pt")
    #     return inputs

    # def forward(self, tensor_dict):
    #     output_tensors = self.model.forward(**tensor_dict)
    #     return {**output_tensors, **tensor_dict}

    # def postprocess(self, tensor_dict):
    #     ## Very Task-specific; see FillMaskPipeline.postprocess
    #     return super().postprocess(tensor_dict)

unmasker = MyMlmPipeline()
unmasker("Hello, Mr. Bert! How is it [MASK]?", verbose=True)

我们可以看到，模型由两个主要的部分组成：

* `tokenizer` ：将输入的字符串转换成模型可用的形式。
* `model` ：负责将输入的张量转换成输出张量的深度学习模型。

借助这些，pipeline 就能通过非常直观的组织模式来支持其精简的接口：

* `preprocess` ：人的直观输入 $\to$ 张量输入。由 `tokenizer` 执行
* `forward` ：张量输入 $\to$ 张量输出。由 `model` 执行
* `postprocess` ：张量输出 $\to$ 人类输出。由 pipeline 任务执行

对于深度学习而言，这一切似乎都相当合理：模型以数字的形式进行推理。不过在语言任务中，您应该不想向大部分用户暴露这一点吧。这种输入输出都是人类语言的模型，可以让用户很容易上手并开始使用。现在，希望您在接触开源 LLM 生态系统时能感觉更自如一些！



---




## 1.4. 您的课程环境

是的，拉取模型就是这么简单！在整个课程中，可以随时拉取您认为有趣的模型，看看它们能做点什么！除非我们明确要求您使用一个巨大的模型，否则请尽量用小型模型；**您的计算环境相对来说很强了，但也不是无限的！** 我们已经为您预加载了几个模型，请在这里进行查看（再看看它们的许可）：[`extras_and_licenses/99_licenses.ipynb`](extras_and_licenses/99_licenses.ipynb) 。

本课程中，您将使用比消费级硬件更加强大的计算资源，对语言建模来说，记住以下几点尤为重要：

* **系统内存**：大型语言模型很占空间，这些模型很容易就会超出消费级显存的大小。某些单个模型可能就需要数十甚至数百 GB 的内存，不过当前环境所配置的容量足以承载我们的内存需求。
* **GPU**：GPU 算力对于执行快速的深度学习训练和推理至关重要，因为深度学习需要对大量的数字进行运算来将您的输入变成预测的结果。大部分计算都是[易并行](https://en.wikipedia.org/wiki/Embarrassingly_parallel)的，现在很多 GPU 数以千计的核心（尤其是NVIDIA GPU 的 [CUDA 核心](https://en.wikipedia.org/wiki/CUDA)）对于加速前向传播和反向传播非常有用。
	+ **GPU 显存**：大型语言模型需要先载入 GPU 以便能快速使用，因此 GPU 显存对于能否放下必要的信息非常重要。许多应用都试图充分利用 CPU 和 GPU，但有时过低的 GPU 显存会严重限制您使用 LLM 加速的能力。

具体来说，您可以使用以下资源：

In [ ]:
%%bash
echo """
===================================================
GPU SPECIFICATION
===================================================
"""
nvidia-smi
echo """
===================================================
MEMORY SPECIFICATION
===================================================
"""
cat /proc/meminfo

**是的，不错的计算资源，但不是无限的！**

在启动下一个 notebook 之前，请运行以下代码单元重启 Jupyter 内核。这将防止之后的 notebook 出现内存不足的问题。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)



---




## 1.5. 总结

您现在已经明白拉取一个语言模型有多简单，下面到了困难部分：

**我可以在实际中使用这些模型吗？** 这取决于模型的许可：

* ***我们先接触的模型会有数据相关的许可，但许多经过微调的模型是仅为了验证想法而训练的，因此不可商用。*** 完成本课程后，您就能试用这些模型，看能不能找到既有权使用又好用的模型。或者，您可以从找到的模型中汲取灵感，用自己的数据集来微调！
* ***我们后面接触的模型是可商用的，并且非常强大和通用！*** 它们本身就非常出色，还能跟较小的模型配合使用，以满足计算资源和控制结构的需求！
* **注意**：我们强烈建议您快速看一下 [`extras_and_licenses/99_licenses.ipynb`](extras_and_licenses/99_licenses.ipynb) 以进一步了解许可和注意事项。

**它们的工作原理和原因是什么？** 我们将详细讨论这一点。

**您该选择什么模型？** 我们将详细介绍这一点。

**在接下来的几个 notebook 中，我们将熟悉这些系统的工作原理！**

<font color="#76b900">**再次欢迎您参加这门课程！！**</font>